In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

In [2]:
Xtrain = pickle.load(open('Xtrain_ANN.data', 'rb'))
Ytrain = pickle.load(open('Ytrain_ANN.data', 'rb'))

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

Using TensorFlow backend.


In [4]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [5]:
earlystop = EarlyStopping(patience=20, restore_best_weights=True)

reduce_learning = ReduceLROnPlateau(patience=5, verbose=1, cooldown=10)

In [6]:
callbacks = [earlystop, reduce_learning]

In [7]:
clf = Sequential()

In [8]:
clf.add(Dense(units=400, activation='relu', input_dim=598))
clf.add(BatchNormalization())
clf.add(Dropout(rate=0.25))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
clf.add(Dense(units=400, activation='relu'))
clf.add(BatchNormalization())
clf.add(Dropout(rate=0.25))

In [10]:
clf.add(Dense(units=1, activation='sigmoid'))

In [11]:
clf.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
clf.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 400)               239600    
_________________________________________________________________
batch_normalization_1 (Batch (None, 400)               1600      
_________________________________________________________________
dropout_1 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 400)               160400    
_________________________________________________________________
batch_normalization_2 (Batch (None, 400)               1600      
_________________________________________________________________
dropout_2 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 401       
Total para

In [13]:
history = clf.fit(Xtrain, Ytrain, epochs=100, batch_size=64, validation_split=0.2, shuffle=True, callbacks=callbacks)

Instructions for updating:
Use tf.cast instead.
Train on 472432 samples, validate on 118108 samples
Epoch 1/100
472432/472432 [==============================] - 40s 85us/step - loss: 0.1223 - acc: 0.9676 - val_loss: 0.2494 - val_acc: 0.9595
Epoch 2/100
472432/472432 [==============================] - 39s 82us/step - loss: 0.1095 - acc: 0.9716 - val_loss: 0.2306 - val_acc: 0.9614
Epoch 3/100
472432/472432 [==============================] - 39s 83us/step - loss: 0.1082 - acc: 0.9721 - val_loss: 0.2328 - val_acc: 0.9616
Epoch 4/100
472432/472432 [==============================] - 39s 82us/step - loss: 0.1074 - acc: 0.9722 - val_loss: 0.2404 - val_acc: 0.9609
Epoch 5/100
472432/472432 [==============================] - 39s 82us/step - loss: 0.1066 - acc: 0.9724 - val_loss: 0.2137 - val_acc: 0.9632
Epoch 6/100
472432/472432 [==============================] - 39s 83us/step - loss: 0.1061 - acc: 0.9725 - val_loss: 0.2778 - val_acc: 0.9591
Epoch 7/100
472432/472432 [===========================

KeyboardInterrupt: 

In [ ]:
clf.evaluate(Xtrain,Ytrain)

In [ ]:
clf.save('clf_ANN_modv3.h5')

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(5, 4))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.figure(figsize=(5, 4))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import log_loss, classification_report, confusion_matrix

In [ ]:
Ytrain_ = clf.predict(Xtrain) > 0.5

In [ ]:
print(log_loss(Ytrain, Ytrain_))
print(confusion_matrix(Ytrain, Ytrain_))
print(classification_report(Ytrain, Ytrain_))

In [ ]:
del(Xtrain)
del(Ytrain)

In [ ]:
Xtest = pickle.load(open('Xtest_ANN.data', 'rb'))

In [ ]:
Ytest_ = clf.predict(Xtest)
del(Xtest)

In [ ]:
Ytest_

In [ ]:
submission = pd.read_csv('./raw/sample_submission.csv', index_col='TransactionID')

In [ ]:
submission.count()

In [ ]:
submission['isFraud'] = Ytest_
submission.head()

In [ ]:
submission.to_csv('predicted_ANN_modv3.csv')

In [ ]:
submission[submission['isFraud'] > 0.5].count()